In [ ]:
# %%capture
# !pip install scispacy negspacy memory_profiler

In [ ]:
import spacy
import scispacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from negspacy.negation import Negex
from scispacy.linking import EntityLinker
import pandas as pd  
import numpy as np
from spacy.lang.en import English
import re
from memory_profiler import profile
import time
import sys
import psutil 
import warnings
from pandas.core.common import SettingWithCopyWarning


warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
extractor_to_use = 5
# Functions needed for entity extraction models
f=open('acronyms.txt','r')
acronyms_dict={}
l=f.readline()
l=f.readline()
while l:
    l1=l.split("\t")
    acronyms_dict[l1[0].lower()]=l1[1].lower()
    l=f.readline()
f.close()

In [ ]:
#lemmatizing the notes to capture all forms of negation(e.g., deny: denies, denying)
def lemmatize(note, nlp):
    doc = nlp(note)
    lemNote = [wd.lemma_ for wd in doc]
    return " ".join(lemNote)

def acronym_to_full_text(row,acronyms_dict):
    final_text = ""
  
    for word in row.split(" "):
        if word.strip() in acronyms_dict.keys():
            final_text=final_text+acronyms_dict[word.strip()]+" "
        else:
            final_text=final_text+word.strip()+" "
    return final_text.strip()

In [ ]:
def preprocessing_matching_algorithm():
   # Input file is this diagnosis one where we are taking one, bi, and ngram values.
   # combination helps in making newer words for each word/s and combination/s.
    Full_data=pd.ExcelFile(r'Diagnosis N Gram MAX 3 12April22upd2.xlsx',engine='openpyxl')

    one_gram=pd.read_excel(Full_data,sheet_name='Unigram')

    one_gram=one_gram.dropna()
    one_gram.rename(columns={'Category(Investigation/Complaint/Diagnosis)':'Category(Investigation/Complaint/Diagnosis/Procedure)'},inplace=True)
    one_gram.dropna(subset=['Label'],inplace=True)
    one_gram=one_gram[['word','Category(Investigation/Complaint/Diagnosis/Procedure)','Label']]


    Bigram=pd.read_excel(Full_data,sheet_name='Bigram')
    Bigram['Label']=np.where(Bigram['Label2'].isna(),Bigram['Label1'],Bigram['Label1']+" "+Bigram['Label2'])
    Bigram.rename(columns={'Category(Investigation/Complaint/Diagnosis)':'Category(Investigation/Complaint/Diagnosis/Procedure)'},inplace=True)
    Bigram.dropna(subset=['Label'],inplace=True)
    Bigram=Bigram[['word','Category(Investigation/Complaint/Diagnosis/Procedure)','Label']]

    
    Trigram=pd.read_excel(Full_data,sheet_name='Trigram')
    Trigram.fillna("",inplace=True)
    Trigram['Label']=Trigram['Label1']+" "+Trigram['Label2']+" "+Trigram['Label3']
    Trigram['Label']=Trigram['Label'].str.strip()
    Trigram['len']=Trigram.Label.str.len()
    Trigram=Trigram[Trigram['len']>0]
    Trigram.dropna(subset=['Label'],inplace=True)
    Trigram=Trigram[['word','Category(Investigation/Complaint/Diagnosis/Procedure)','Label']]
    
    n_gram_sheet=one_gram.append(Bigram)
    n_gram_sheet=n_gram_sheet.append(Trigram)
    n_gram_sheet.fillna("",inplace=True)
    n_gram_sheet=n_gram_sheet.replace('nan',"")
    n_gram_sheet['len']=n_gram_sheet['Category(Investigation/Complaint/Diagnosis/Procedure)'].str.len()
    n_gram_sheet=n_gram_sheet[n_gram_sheet['len']>0]
    n_gram_sheet['label']=n_gram_sheet['Category(Investigation/Complaint/Diagnosis/Procedure)']+":"+n_gram_sheet['Label']
    n_gram_sheet.rename(columns={'word':'pattern'},inplace=True)
    
    investigation=pd.read_excel('UnifiedBiomarkerMapping_1.xlsx',engine='openpyxl',sheet_name='Top TestNames')
    investigation.dropna()
    investigation.rename(columns={'lab test names':'pattern'},inplace=True)
    investigation['pattern']=investigation['pattern'].astype(str)

    investigation['label']='Investigation:'+(investigation['pattern'])
    n_gram_sheet=n_gram_sheet.append(investigation)

    drugs=pd.read_csv('final_drug_list.csv')
    drugs.dropna()
    drugs.rename(columns={'value_upd_3':'pattern'},inplace=True)
    drugs['label']='Drugs:'+drugs['pattern']
    print(drugs)
    n_gram_sheet=n_gram_sheet.append(drugs)
    n_gram_sheet=n_gram_sheet[['label','pattern']]

    gram_dict=n_gram_sheet.to_dict("record")
  
    length_gram_dict = len(gram_dict)
    for i in range(0,length_gram_dict):
        pattern = gram_dict[i]['pattern']
        pattern = acronym_to_full_text(pattern,acronyms_dict)
        pattern_list = pattern.split()
        l=[]
        l1 = []
        for j in range(0,len(pattern_list)):
            d={}
            d["LOWER"]=pattern_list[j]
            l.append(d)
            l1.append(d)
            if j!=len(pattern_list)-1:
                l1.append({"IS_PUNCT": True})
        gram_dict.append({}) 
        gram_dict[i]['pattern'] = l
        
        gram_dict[i+length_gram_dict]['pattern'] = l1
        gram_dict[i+length_gram_dict]['label'] = gram_dict[i]['label']
    #print(gram_dict)
    return gram_dict

In [ ]:
#Reference https://lhncbc.nlm.nih.gov/ii/tools/MetaMap/Docs/SemanticTypes_2018AB.txt
valid_semantic_types = ['T109','T116','T020','T190','T017','T195','T053','T038','T030','T019','T200','T060','T047','T129','T037','T059','T034','T048','T046','T121','T061','T184','T033','T191' ]
complaint_semantic_types = ['T184','T033']
diagnosis_semantic_types = ['T020','T190','T053','T038','T029','T023','T030','T019','T047','T129','T037','T048','T046','T191']
investigation_semantic_types=['T017','T034','T060','T059','T061']
procedure_semantic_types=[]
drugs_semantic_types = ['T195','T200','T116','T109','T122']

In [ ]:
patterns = preprocessing_matching_algorithm()

if extractor_to_use==1 or extractor_to_use == 3:
    nlp1 = spacy.load("en_core_sci_lg")

    nlp1.add_pipe("scispacy_linker", config={"linker_name": "umls","max_entities_per_mention": 1,"threshold": 0.9})
    nlp1.add_pipe("negex",config={"chunk_prefix": ["no"]})
if extractor_to_use == 2 or extractor_to_use == 3:
    nlp2 = spacy.load("en_ner_bc5cdr_md")
    nlp2.add_pipe("negex",config={"chunk_prefix": ["no"]})
if extractor_to_use == 4:
    nlp3 = English()
    ruler = nlp3.add_pipe("entity_ruler", config={"overwrite_ents":True})
    ruler.add_patterns(patterns)
    nlp3.add_pipe("negex",config={"chunk_prefix": ["no"]})

       Unnamed: 0        pattern                label
0               0  betacap tr 40  Drugs:betacap tr 40
1               1      ferium xt      Drugs:ferium xt
2               2    pexep cr 12    Drugs:pexep cr 12
3               3         forcan         Drugs:forcan
4               4     disperzyme     Drugs:disperzyme
...           ...            ...                  ...
95038       85570       Puron 12       Drugs:Puron 12
95039       85590     Sinarest 0     Drugs:Sinarest 0
95040       85756        Entof 0        Drugs:Entof 0
95041       85859     Diflucor 2     Drugs:Diflucor 2
95042       85995       Biomus 0       Drugs:Biomus 0

[95043 rows x 3 columns]


In [ ]:
# %%capture
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz

In [ ]:
# RG Files -> SVAAS -> HC Folder -> Complain List

In [ ]:
if extractor_to_use == 5 or extractor_to_use == 7:
    nlp4 = spacy.load("en_core_sci_lg")
    nlp4.add_pipe("scispacy_linker", config={"linker_name": "umls","max_entities_per_mention": 1,"threshold": 0.9})
    ruler1 = nlp4.add_pipe("entity_ruler", config={"overwrite_ents":True})
    ruler1.add_patterns(patterns)
    print("Ruler 1: ", ruler1)
    nlp4.add_pipe("negex",config={"chunk_prefix": ["no"]})

if extractor_to_use == 6 or extractor_to_use == 7:
    nlp5 = spacy.load("en_ner_bc5cdr_md")
    ruler2 = nlp5.add_pipe("entity_ruler", config={"overwrite_ents":True})
    ruler2.add_patterns(patterns)
    nlp5.add_pipe("negex",config={"chunk_prefix": ["no"]})

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpwggnt9pr
Finished download, copying /tmp/tmpwggnt9pr to cache at /root/.scispacy/datasets/e9f7327283e43f0482f7c0c71b71dec278a58ccb3ffdd03c2c2350159e7ef146.f2a350ad19015b2591545f7feeed6a6d6d2fffcd635d868a5d7fc0dfc3cadfd8.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmp6l9vi79_
Finished download, copying /tmp/tmp6l9vi79_ to cache at /root/.scispacy/datasets/f48455d6c79262057cce66b4619123c2b558b21092d42fac97f47bb99a5b8f9f.dd70d3dffe7d90d7ac8914460e16a48375dab32485fb6313a34e6fbcaf53218b.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmpmm01jji1
Finished download, copying /tmp/tmpmm01jji1 to cache at /root/.scispacy/da

In [ ]:
def entity_extraction(row,value):
 #   print(row[value])
    val_list =row[value].split("##")
   # print(val_list)
    complaint = []
    diagnosis = []
    investigation=[]
    procedure = []
    drugs=[]
        
    for val in val_list:
        regexpattern = r'\.\.+'
        val = re.sub(regexpattern, ' ', val)
        val  = ' '.join(acronym_to_full_text(val.strip().replace('-',' ').replace('/',' ').replace('+',' ').strip(),acronyms_dict).split())

        #print("val is ",val)
            
        if extractor_to_use==1 or extractor_to_use == 3:
            doc1 = nlp1(val)
        if extractor_to_use==2 or extractor_to_use ==3 :
            doc2 = nlp2(val)
        if extractor_to_use==4:
            doc3 = nlp3(val)
        if extractor_to_use==5 or extractor_to_use==7:
            doc4 = nlp4(val)
        if extractor_to_use==6 or extractor_to_use==7:
            doc5=nlp5(val)
        
       
        if extractor_to_use == 4 or extractor_to_use==5 or extractor_to_use==6 or extractor_to_use==7:
            
            label=[]
            text=[]
            ent_list=[]
            if extractor_to_use ==4:
                ent_list.append(doc3.ents)
            if extractor_to_use == 5 or extractor_to_use==7:
                ent_list.append(doc4.ents)
            if extractor_to_use==6 or extractor_to_use == 7:
                ent_list.append(doc5.ents)
            
            for ent_tuple in ent_list:
                for ent in ent_tuple:
                   # print(ent,ent.label_,ent.text,ent._.negex)
                    if ent._.negex == 0 and ":" in ent.label_:
                        label.append(str(ent.label_))
                        text.append(ent.text)
            
            if len(label) > 0:
                all_labels=pd.DataFrame(label,columns =['Names'])
                all_labels[['Category', 'insight']] = all_labels['Names'].str.split(':', 1, expand=True)
                all_labels['Category']=all_labels.Category.str.strip()
                all_labels['insight']=all_labels.insight.str.strip()
                final_label=all_labels[['Category','insight']].groupby(['Category'])['insight'].apply(','.join).reset_index()
                #print(final_label)
                for i in range(len(final_label)):    
                    if final_label.iloc[i]['Category']=='Complaint':
                        if final_label.iloc[i]['insight'] not in complaint:
                            complaint.append(final_label.iloc[i]['insight'])
                    elif final_label.iloc[i]['Category']=='Investigation':
                        if final_label.iloc[i]['insight'] not in investigation:
                            investigation.append(final_label.iloc[i]['insight'])
                    elif final_label.iloc[i]['Category']=='Diagnosis':
                        if final_label.iloc[i]['insight'] not in diagnosis:
                            diagnosis.append(final_label.iloc[i]['insight'])
                    elif final_label.iloc[i]['Category']=='Drugs':
                        if final_label.iloc[i]['insight'] not in drugs:
                            drugs.append(final_label.iloc[i]['insight'])
                    elif final_label.iloc[i]['Category']=='Procedure':
                        if final_label.iloc[i]['insight'] not in procedure:
                            procedure.append(final_label.iloc[i]['insight']) 
          
        if extractor_to_use ==1 or extractor_to_use == 3 or extractor_to_use == 5 or extractor_to_use==7:
            if extractor_to_use ==1 or extractor_to_use == 3:
                ent_list = doc1.ents
                linker = nlp1.get_pipe("scispacy_linker")
            else:
                ent_list = doc4.ents
                linker = nlp4.get_pipe("scispacy_linker")
            #print(ent_list)
            for entity in ent_list:
                #print(entity.text,entity.label_,entity._.negex,entity._.kb_ents)
                if entity._.negex == 0 and entity.label_=='ENTITY':
                #Link to UMLS knowledge baseT048
                
             #   print(entity._.kb_ents)
                    for kb_entry in entity._.kb_ents:
                        sem_type=linker.kb.cui_to_entity[kb_entry[0]].types[0]
                        #print(entity,sem_type,linker.kb.cui_to_entity[kb_entry[0]].types)
                        #print(linker.kb.cui_to_entity[kb_entry[0]].canonical_name)
                        if sem_type in valid_semantic_types:
                            if sem_type in complaint_semantic_types:
                                if str(entity.text) not in complaint:
                                    complaint.append(str(entity.text))
                            elif sem_type in diagnosis_semantic_types:
                                if str(entity.text) not in diagnosis:
                                    diagnosis.append(str(entity.text))
                            elif sem_type in investigation_semantic_types:
                                if str(entity.text) not in investigation:
                                    investigation.append(str(entity.text))
                            elif sem_type in procedure_semantic_types:
                                if str(entity.text) not in procedure:
                                    procedure.append(str(entity.text))
                            elif sem_type in drugs_semantic_types:
                                if str(entity.text) not in drugs:
                                    drugs.append(str(entity.text))
            
                
        if extractor_to_use==2 or extractor_to_use ==3 or extractor_to_use==6 or extractor_to_use==7:
            if extractor_to_use == 2 or extractor_to_use == 3:
                ent_list = doc2.ents
            else:
                ent_list = doc5.ents
            for entity in ent_list:
                if entity._.negex == 0:
                    if entity.label_ == 'DISEASE':
                        if str(entity.text) not in diagnosis and str(entity.text) not in complaint:
                            diagnosis.append(str(entity.text))
                    elif entity.label_ == 'CHEMICAL':
                        if str(entity.text) not in drugs:
                            drugs.append(str(entity.text))
        
       
                    
     
       
    row['COMPLAINT'] = ",".join(complaint)
    row['DIAGNOSIS'] = ",".join(diagnosis)
    row['INVESTIGATION'] = ",".join(investigation)
    row['PROCEDURE'] = ",".join(procedure)
    row['DRUGS'] = ','.join(drugs)
    
        
    return row

In [ ]:
def clean_data(row):
    rowlist = row.split(",")
    modrowlist=[]
    for r in rowlist:
        if r.strip() not in modrowlist and r.strip()!="":
            modrowlist.append(r.strip())
    return ",".join(modrowlist)

In [ ]:
def generateResults():
    data2 = pd.read_csv("dt.csv")
    data2.fillna("NA", inplace=True)

    combined_values = data2['complaints'].values + "," + data2['advice'].values + "," + data2['diagnosis'].values

    data_req = pd.DataFrame(np.c_[data2['brdg_appt_conslt_sid'].values, combined_values], 
                              columns=['brdg_appt_consult_id', 'combined_diagnosis']
                            )
    
    data_req['combined_diagnosis'] = data_req['combined_diagnosis'].apply(lambda x: x.lower())
    data = data_req.copy()
    del data_req
    
    data['combined_diagnosis']=data['combined_diagnosis'].astype(str)
    data.replace("", "na", inplace=True)
    print(data.shape)
    data.dropna(how='all',axis=0,inplace=True)
    data.dropna(how='all', axis=1, inplace=True)
    print(data.shape)
    df_ext_Remarks=data.apply(lambda row:entity_extraction(row,'combined_diagnosis'),axis=1)
    df_ext_Remarks.to_csv("remarks.csv", index=False)
    return df_ext_Remarks

In [26]:
def generateMappedComplainDataFrame(complain_data, original_data):
    df_result = generateResults()
    df_result.fillna("NaN", inplace=True)
    df_result['DIAGNOSIS'] = df_result['DIAGNOSIS'].apply(lambda x: x.lower())
    
    df_result["DIAGNOSIS"] = df_result["DIAGNOSIS"].str.split(",")
    df_result = df_result.explode("DIAGNOSIS")
    
    assert ".csv" in complain_data, "Make sure to insert a csv file."
    assert ".csv" in original_data, "Make sure to insert a csv file."
    data_req = pd.read_csv(complain_data)
    
    brdg = pd.read_csv(original_data)
    df_result.rename(columns={'brdg_appt_consult_id':'brdg_appt_conslt_sid'}, inplace=True)
    
    cols = ["brdg_appt_conslt_sid", "DIAGNOSIS", "COMPLAINT"]

    df_result = df_result[cols]
    df_result.columns = ['brdg_appt_conslt_sid', 'mapped_DIAGNOSIS', 'mapped_COMPLAINT']
    
    brdg.merge(df_result, on="brdg_appt_conslt_sid", how="left").to_csv("merged_csv_complain_v2.csv", index=False)

    df_final = pd.read_csv("merged_csv_complain_v2.csv")
    print("Final dataframe has been saved with the name of output.csv")
    df_final.replace("nan", "").fillna(" ").to_csv("output.csv", index=False)
    
    df_req = pd.read_csv(complain_data)
    
    df_final= df_final.merge(df_req, left_on=["mapped_DIAGNOSIS"], right_on=["Words"], how="left")
    
    return df_final.replace("nan", "").fillna(" ")
    
    

In [ ]:
original_data = pd.read_csv("dt.csv")
complain_data = pd.read_csv("full_complain_list.csv")


df_final = generateMappedComplainDataFrame(complain_data, original_data)

In [22]:
df_final.head()

,brdg_appt_conslt_sid,appointment_id,consultation_id,appointment_dt_sid,appointment_datetime,appointment_status,appointment_type,svaas_appointment_id,vendor_sid,doctor_id,...,bp_sys,pulse,weight,height,bmi,row_insert_dt,row_update_dt,consultation_doc_ref,mapped_DIAGNOSIS,mapped_COMPLAINT
0,1,2253,-1,20220208,2/8/2022 19:45,cancelled,InPerson,D-HX0802-1146-4247758236,1239,584,...,NaN,NaN,NaN,NaN,NaN,9/7/2022 14:30,9/7/2022 14:30,NaN,NaN,NaN
1,2,4551,-1,20220816,8/16/2022 10:10,declined,OnlineConsultation,D-HX1608-0942-2283404928,1790,1233,...,NaN,NaN,NaN,NaN,NaN,9/7/2022 14:30,9/7/2022 14:30,NaN,NaN,NaN
2,3,3731,899,20220525,5/25/2022 19:00,completed,OnlineConsultation,D-HX2505-1302-3731620448,726,1247,...,NaN,NaN,NaN,NaN,NaN,9/7/2022 14:30,9/7/2022 14:30,1653486348_pdf_Psn4t6MIZJAFW3uYoR7e.pdf,htn,NaN
3,4,2241,-1,20220207,2/7/2022 18:00,completed,InPerson,D-HX0702-1021-6989648006,849,1107,...,NaN,NaN,NaN,NaN,NaN,9/7/2022 14:30,9/7/2022 14:30,NaN,NaN,NaN
4,5,2586,591,20220307,3/7/2022 17:30,completed,InPerson,D-HX0703-0956-1922150952,1838,645,...,NaN,NaN,NaN,NaN,NaN,9/7/2022 14:30,9/7/2022 14:30,1646656625_pdf_t9oIOr8FNXGkmLnE245S.pdf,pityriasis,NaN
